# **Cài đặt môi trường**

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 17.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (p

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
# Load model bloomz-1b1
model = AutoModelForCausalLM.from_pretrained("bigscience/bloomz-1b1").to("cuda")
# Load tokenizer từ bloomz-1b1
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-1b1")

# **Tải dataset hỏi đáp web bán hàng**

In [ ]:
!wget https://github.com/phatjkk/data/raw/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx

--2023-06-24 10:22:17--  https://github.com/phatjkk/data/raw/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx [following]
--2023-06-24 10:22:18--  https://raw.githubusercontent.com/phatjkk/data/main/LLM/Ecommerce_FAQ_Chatbot_dataset_vi.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12116 (12K) [application/octet-stream]
Saving to: ‘Ecommerce_FAQ_Chatbot_dataset_vi.xlsx’

Ecommerce_FAQ_Chatb 100%[===================>]  11.83K  --.-KB/s    in 0s      

2023-06-24 10:22:18 (90.0 MB/s)

In [ ]:
import pandas as pd
df = pd.read_excel("Ecommerce_FAQ_Chatbot_dataset_vi.xlsx", header=None,skiprows=1).drop(0)

In [ ]:
df

,0,1
1,Làm thế nào tôi có thể tạo một tài khoản?,"Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ..."
2,Phương thức thanh tóan nào được chấp nhận?,"Chúng tôi chấp nhận các thẻ tín dụng chính, t..."
3,Làm thế nào tôi có thể theo dõi đơn hàng của ...,Bạn có thể theo dõi đơn đặt hàng của mình bằn...
4,Chính sách hoàn trả của bạn là gì?,Chính sách hoàn trả của chúng tôi cho phép bạ...
5,Tôi có thể hủy đơn hàng của mình không?,Bạn có thể hủy đơn đặt hàng nếu nó chưa được v...
...,...,...
75,Tôi có thể đặt hàng nếu sản phẩm được liệt kê...,"Nếu một sản phẩm được liệt kê là ""hết hàng"" n..."
76,Tôi có thể trả lại sản phẩm nếu nó được mua b...,"Có, bạn có thể trả lại sản phẩm đã mua bằng t..."
77,Tôi có thể yêu cầu một sản phẩm nếu nó hiện kh...,Nếu một sản phẩm không có sẵn màu sắc ưa thíc...
78,Tôi có thể đặt hàng nếu sản phẩm đó được liệt...,"Nếu một sản phẩm được liệt kê là ""sắp ra mắt""..."


In [ ]:
from datasets import load_dataset
from datasets import Dataset
Dataset.from_pandas(df)[0]

{'0': ' Làm thế nào tôi có thể tạo một tài khoản?',
 '1': " Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ở góc trên cùng bên phải của trang web của chúng tôi và làm theo hướng dẫn để hoàn tất quy trình đăng ký."}

In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
# Kết hợp câu hỏi và câu trả lời liền kề nhau để model học nội dung
# Mục đích là để khi đưa 1 đoạn câu hỏi, model sẽ tự nhiểu và gen ra câu trả lời
# phía sau
def merge_columns(example):
    example["prediction"] = example["0"] + " ->: " + str(example["1"])
    return example

In [ ]:
# Map function trên vào dataset
ds = ds.map(merge_columns)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
# Tokenizer/Véc tơ hóa văn bản (Chuyển văn bản thành số để training)
def tokeni(example):
    example["prediction_token"] = tokenizer(example["prediction"], return_tensors='pt', padding=True)['input_ids']
    return example
# Map function trên vào dataset
ds = ds.map(tokeni,batched=True)

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [ ]:
# Hàm tính số lượng parameters có thể train được trên model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
# Tạo LoraConfig để áp dụng Lora vào việc training mô hình
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8, #attention heads
    lora_alpha=16, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('//172.28.0.1'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

trainable params: 1179648 || all params: 1066493952 || trainable%: 0.11060990995662018


In [ ]:
# Training model bằng hàm trainer
import transformers
trainer = transformers.Trainer(
    model=model,
    train_dataset=ds["prediction_token"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        warmup_steps=100,
        max_steps=150,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs',
        # remove_unused_columns=False
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!
trainer.train()

Step,Training Loss
1,1.573800
2,1.135200
3,1.227400
4,1.445500
5,0.789900
6,1.321400
7,1.250200
8,1.175600
9,1.275200
10,1.447900


TrainOutput(global_step=150, training_loss=1.0087280972798665, metrics={'train_runtime': 19.642, 'train_samples_per_second': 15.273, 'train_steps_per_second': 7.637, 'total_flos': 101958824411136.0, 'train_loss': 1.0087280972798665, 'epoch': 3.75})

In [ ]:
tokenizer

BloomTokenizerFast(name_or_path='bigscience/bloomz-1b1', vocab_size=250680, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False)

In [ ]:
print(ds["0"][0])
print(ds["1"][0])

 Làm thế nào tôi có thể tạo một tài khoản?
 Để tạo tài khoản, hãy nhấp vào nút 'Đăng ký' ở góc trên cùng bên phải của trang web của chúng tôi và làm theo hướng dẫn để hoàn tất quy trình đăng ký.


In [ ]:
question = str(ds["0"][0]+" ->: ")
question

' Làm thế nào tôi có thể tạo một tài khoản? ->: '

In [ ]:
# In ra kết quả
inputs = tokenizer( question, return_tensors="pt")
# Do biến inputs được lưu trên cpu mà model thì load trên gpu nên phải chuyển
# biến này lên gpu bằng hàm .to("cuda")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Tôi có thể tạo tài khoản bằng cách nào? ->:  Bạn có thể tạo tài khoản bằng cách:  - Đăng ký bằng tài khoản hiện có của bạn - Đăng ký bằng tài khoản mới - Đăng ký bằng tài khoản bằng email Đăng ký bằng tài khoản bằng email. Bạn có thể đăng ký bằng tài khoản email của mình để tạo tài khoản. Nhấp vào nút "Đăng ký" ở góc trên bên phải trang để bắt đầu tạo tài khoản. Bạn có thể đăng ký bằng tài khoản email của mình để tạo tài khoản.


In [ ]:
question="Làm sao để mua hàng?"
inputs = tokenizer( question, return_tensors="pt")
with torch.autocast("cuda"):
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

Làm sao để mua hàng? ->:  Chúng tôi cung cấp các tùy chọn mua hàng để bạn có thể mua hàng với nhiều lựa chọn hơn. Chúng tôi hy vọng rằng bạn sẽ tìm được phương án phù hợp với mình. Nếu cần hỗ trợ, vui lòng liên hệ với nhóm hỗ trợ khách hàng của chúng tôi. Chúng tôi sẽ hỗ trợ bạn trong quá trình tìm hiểu và mua hàng.
